#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-02 13:39:05--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  9.14MB/s    in 0.5s    

2025-02-02 13:39:06 (9.14 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [ ]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [ ]:
# прочитаем описание README
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией
         список идентификаторов пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификатор пользователя | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
        быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

'\nПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ\n==============================================\n\nВот краткое описание данных.\n\nml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:\n                gunzip ml-data.tar.gz\n                tar xvf ml-data.tar\n                mku.sh\n\nu.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.\n         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы\n         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.\n         Это разделенный табуляцией \n         список идентификаторов пользователей | элементов | оценок | временных меток. \n         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix   \n\nu.info - Количество пользователей, элементов и оценок в наборе данных u.\n\nu.item - Информация о фильмах, которые будут показаны в этом разделе;\n         список фильмов, которые будут показаны в виде вкладок: \n

In [ ]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data', delimiter='\t')
u_data.columns = ['userID', 'movieID', 'rating', 'time']
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')

# так читать не хочет
# u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
# попробуем по-другому
import csv
csv_file = open('/content/unpacked_files/ml-100k/u.item', newline='', encoding='latin-1')
u_item = pd.read_csv(csv_file, delimiter='|')
u_item.columns = ['movieID', 'movie_title', 'release_date', 'video_release',
                  'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                  'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                  'Thriller', 'War', 'Western']
u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre', delimiter='|')
u_genre.columns = ['genre', '#']
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user', delimiter='|')
u_user.columns = ['userID', 'age', 'gender', 'occupation', 'p_code']
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')
u_occupation.columns = ['occupation']

In [ ]:
# пользователь | элемент | рейтинг | временная метка
u_data.head()

,userID,movieID,rating,time
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [ ]:
u_info.head()

,943 users
0,1682 items
1,100000 ratings


In [ ]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
u_item_2 = u_item.copy()
u_item_2.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def genre_to_string (str, num):
  if num == 1:
    string = str

  return

In [ ]:
 # добавим единую колонку жанров
 cols = ['unknown', 'Action', 'Adventure', 'Animation', 'Children_s', 'Comedy',
         'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
         'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for index, row in u_item_2.iterrows():
    string = str()
    for ele in cols:
        if row[ele] == 1:
            if string == '':
                string += ele
            else:
                string += ('|'+ele)
        else:
            continue
    u_item.loc[index, 'genre'] = string

In [ ]:
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action|Adventure|Thriller
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action|Comedy|Drama
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime|Drama|Thriller
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama


In [ ]:
# посмотрим на список жанров
u_genre

,genre,#
0,Action,1
1,Adventure,2
2,Animation,3
3,Children's,4
4,Comedy,5
5,Crime,6
6,Documentary,7
7,Drama,8
8,Fantasy,9
9,Film-Noir,10


In [ ]:
# пользователь | возраст | пол | род занятий | почтовый индекс
u_user.head()

,userID,age,gender,occupation,p_code
0,2,53,F,other,94043
1,3,23,M,writer,32067
2,4,24,M,technician,43537
3,5,33,F,other,15213
4,6,42,M,executive,98101


In [ ]:
# профессии
u_occupation.head()

,occupation
0,artist
1,doctor
2,educator
3,engineer
4,entertainment


##LightFM (default)

In [ ]:
import numpy as np

In [ ]:
!pip install lightfm

In [ ]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [ ]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [ ]:
data = fetch_movielens(min_rating=5.0)
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [ ]:
train = movielens['train']
test = movielens['test']

Посмотрим на все это

In [ ]:
train.toarray(), len(train.toarray())

(array([[5, 3, 4, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [5, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 5, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [ ]:
test.toarray(), len(test.toarray())

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [ ]:
data = fetch_movielens()
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>


In [ ]:
# зададим рейтинг
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.44, test 0.13.


In [ ]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.60, test 0.22.


In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [ ]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        S
        E
User 25
     Known positives:
        T
        T
        D
     Recommended:
        S
        L
        C
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        A
        S


##Соберем свой датафрейм

In [ ]:
# объединим u_data и u_user
df_1 = u_data.merge(u_user, on='userID').reset_index(drop=True)
df_1.head()

,userID,movieID,rating,time,age,gender,occupation,p_code
0,186,302,3,891717742,39,F,executive,00000
1,22,377,1,878887116,25,M,writer,40206
2,244,51,2,880606923,28,M,technician,80525
3,166,346,1,886397596,47,M,educator,55113
4,298,474,4,884182806,44,M,executive,01581


In [ ]:
# объединим df_1 и u_item
df_2 = df_1.merge(u_item, on='movieID').reset_index(drop=True)
df_2.head()

,userID,movieID,rating,time,age,gender,occupation,p_code,movie_title,release_date,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,4,884182806,44,M,executive,01581,Dr. Strangelove or: How I Learned to Stop Worr...,01-Jan-1963,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [ ]:
# выберем самое интересное
cols = ['userID', 'movieID', 'movie_title', 'rating', 'age', 'gender', 'occupation',
        'unknown', 'Action', 'Adventure', 'Animation',
        'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
        'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
        'Thriller', 'War', 'Western', 'genre']
df_data = df_2[cols]
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [ ]:
# проверим жанр unknown
df_data['unknown'].sum()

9

In [ ]:
# посмотрим на эти строки
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,
3732,5,267,unknown,4,33,F,other,1,0,0,...,0,0,0,0,0,0,0,0,0,
7151,268,267,unknown,3,24,M,engineer,1,0,0,...,0,0,0,0,0,0,0,0,0,
8465,181,1373,Good Morning (1971),1,26,M,executive,1,0,0,...,0,0,0,0,0,0,0,0,0,
12332,297,267,unknown,3,29,F,educator,1,0,0,...,0,0,0,0,0,0,0,0,0,
14595,319,267,unknown,4,38,M,programmer,1,0,0,...,0,0,0,0,0,0,0,0,0,
48878,532,267,unknown,3,20,M,student,1,0,0,...,0,0,0,0,0,0,0,0,0,
92840,833,267,unknown,1,34,M,writer,1,0,0,...,0,0,0,0,0,0,0,0,0,
99000,422,267,unknown,4,26,M,entertainment,1,0,0,...,0,0,0,0,0,0,0,0,0,


In [ ]:
# считаю целесообразным удалить эти строки
df_data.drop(df_data.index[df_data['unknown'] == 1], axis=0, inplace=True)

<ipython-input-214-eb2aae18e79a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.drop(df_data.index[df_data['unknown'] == 1], axis=0, inplace=True)


In [ ]:
# проверим
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre


In [ ]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())
# посмотрим
list_occupations_unique,  len(list_occupations_unique)

(['executive',
  'writer',
  'technician',
  'educator',
  'engineer',
  'librarian',
  'programmer',
  'administrator',
  'student',
  'retired',
  'other',
  'doctor',
  'marketing',
  'artist',
  'lawyer',
  'salesman',
  'homemaker',
  'healthcare',
  'none',
  'entertainment',
  'scientist'],
 21)

In [ ]:
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [ ]:
#соберем объект жанров по фильмам
series_genre_movies = df_data['genre'].str.split('|')
# посмотрим
series_genre_movies[0], series_genre_movies, len(series_genre_movies)

(['Crime', 'Film-Noir', 'Mystery', 'Thriller'],
 0        [Crime, Film-Noir, Mystery, Thriller]
 1                         [Children_s, Comedy]
 2               [Drama, Romance, War, Western]
 3                               [Crime, Drama]
 4                                [Sci-Fi, War]
                          ...                  
 99271                                 [Comedy]
 99272                         [Comedy, Sci-Fi]
 99273                               [Thriller]
 99274                     [Children_s, Comedy]
 99275                                [Western]
 Name: genre, Length: 99276, dtype: object,
 99276)

##Более интересная модель

In [ ]:
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from lightfm.datasets import fetch_movielens

In [ ]:
# посмотрим на эти строки
df_data.loc[df_data['userID'] == 2144]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre


In [ ]:
df_data.reset_index()

,index,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,1,22,377,Heavyweights (1994),1,25,M,writer,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99262,99271,880,476,"First Wives Club, The (1996)",3,13,M,student,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
99263,99272,716,204,Back to the Future (1985),5,36,F,administrator,0,0,...,0,0,0,0,0,1,0,0,0,Comedy|Sci-Fi
99264,99273,276,1090,Sliver (1993),1,21,M,student,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
99265,99274,13,225,101 Dalmatians (1996),2,47,M,educator,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy


In [ ]:
df_data.head(2147)

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,239,10,Richard III (1995),5,39,M,artist,0,0,0,...,0,0,0,0,0,0,0,1,0,Drama|War
2143,238,111,"Truth About Cats & Dogs, The (1996)",4,42,F,administrator,0,0,0,...,0,0,0,0,1,0,0,0,0,Comedy|Romance
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,unknown
2145,90,662,Somewhere in Time (1980),5,60,M,educator,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance


In [ ]:
series_genre.isna().sum()

0

In [ ]:
# соберем список уникальных жанров
series_genre = df_data['genre'].str.split('|')
list_genre_unique = list(set(np.concatenate(series_genre).ravel()))

list_genre_unique[:10], len(list_genre_unique)

(['Sci-Fi',
  'Western',
  'War',
  'Drama',
  'Children_s',
  'Animation',
  'Action',
  'Crime',
  'Film-Noir',
  'Adventure'],
 19)

In [ ]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())

list_occupations_unique[:5], len(list_occupations_unique)

(['executive', 'writer', 'technician', 'educator', 'engineer'], 21)

In [ ]:
# соберем датасет
dataset = Dataset()
dataset.fit(users=df_data['userID'],
            items=df_data['movieID'],
            item_features=list_genre_unique,
            user_features=list_occupations_unique)

In [ ]:
# соберем список уникальных пользователей и их профессии
df_unique_userID = df_data.drop_duplicates('userID')
list_user_features = [(x,[y]) for x,y in zip(df_unique_userID['userID'],
                      df_unique_userID['occupation'])
]

list_user_features[:10], len(list_user_features)

([(186, ['executive']),
  (22, ['writer']),
  (244, ['technician']),
  (166, ['educator']),
  (298, ['executive']),
  (115, ['engineer']),
  (253, ['librarian']),
  (305, ['programmer']),
  (6, ['executive']),
  (62, ['administrator'])],
 942)

In [ ]:
# соберем список уникальных фильмов и их жанры
df_unique_movieID = df_data.drop_duplicates('movieID')
list_item_features = [(x,y) for x,y in zip(df_unique_movieID['movieID'], series_genre_movies)]

list_item_features[:10], len(list_item_features)

([(302, ['Crime', 'Film-Noir', 'Mystery', 'Thriller']),
  (377, ['Children_s', 'Comedy']),
  (51, ['Drama', 'Romance', 'War', 'Western']),
  (346, ['Crime', 'Drama']),
  (474, ['Sci-Fi', 'War']),
  (265, ['Action', 'Thriller']),
  (465, ['Adventure', 'Children_s', 'Romance']),
  (451, ['Comedy', 'Musical', 'Romance']),
  (86, ['Drama']),
  (257, ['Action', 'Adventure', 'Comedy', 'Sci-Fi'])],
 1681)

In [ ]:
# подготовим sparse matrix
sm_user_features = dataset.build_user_features(list_user_features)
sm_user_features

<942x963 sparse matrix of type '<class 'numpy.float32'>'
	with 1884 stored elements in Compressed Sparse Row format>

In [ ]:
# подготовим sparse matrix
sm_item_features = dataset.build_item_features(list_item_features)
sm_item_features

<1681x1700 sparse matrix of type '<class 'numpy.float32'>'
	with 5227 stored elements in Compressed Sparse Row format>

In [ ]:
# подготовим данные о взаимодействии
sm_interactions, sm_weights = dataset.build_interactions(df_data[['userID',
                                                                  'movieID',
                                                                  'rating']].values)
sm_interactions

<942x1681 sparse matrix of type '<class 'numpy.int32'>'
	with 99276 stored elements in COOrdinate format>

Вместо использования исходных идентификаторов пользователей и элементов в нашем наборе данных LightFM внутренне присваивает каждому пользователю и элементу новый последовательный неотрицательный целочисленный идентификатор. Мы можем увидеть это сопоставление следующим образом:

In [ ]:
user_id_map, user_feature_map, item_id_map, feature_item_map = dataset.mapping()

Посмотрим

In [ ]:
list(user_id_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [ ]:
list(user_feature_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [ ]:
list(item_id_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [ ]:
list(feature_item_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [ ]:
# оценка эффективности
sm_train_interactions, sm_test_interactions = random_train_test_split(sm_interactions,
                                                                      test_percentage=0.2,
                                                                      random_state=42)
print(f'Shape of train interactions: {sm_train_interactions.shape}')
print(f'Shape of test interactions: {sm_test_interactions.shape}')

Shape of train interactions: (942, 1681)
Shape of test interactions: (942, 1681)


In [ ]:
train.shape

(943, 1682)

In [ ]:
test.shape

(943, 1682)

In [ ]:
# построим модель
LEARNING_RATE = 0.05
EPOCHS = 10
COMPONENTS = 10
ITEM_ALPHA = 1e-6   # коэффициент упорядочения характеристик признаков
USER_ALPHA = 1e-6   # коэффициент упорядочения пользовательских функций

model = LightFM(loss='warp',
                no_components=COMPONENTS,
                learning_rate=LEARNING_RATE,
                item_alpha=ITEM_ALPHA,
                user_alpha=USER_ALPHA,
                random_state=42)

model.fit(interactions=sm_train_interactions,
          user_features=sm_user_features,
          item_features=sm_item_features,
          epochs=EPOCHS)

In [ ]:
# оценим точность
train_precision = precision_at_k(model,
                                 test_interactions=sm_train_interactions,
                                 user_features=sm_user_features,
                                 item_features=sm_item_features,
                                 k=10).mean()



test_precision = precision_at_k(model,
                                test_interactions=sm_test_interactions,
                                user_features=sm_user_features,
                                item_features=sm_item_features,
                                k=10
                                ).mean()

train_precision, test_precision

(0.40562633, 0.105944805)

##Установка TensorFlow

In [2]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorboard
    Found existing installation

In [3]:
!pip install tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.9 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np

##Анализ датасета

In [49]:
# загрузим MovieLens
ratings, ratings_info = tfds.load('movielens/100k-ratings',
                                     with_info=True,
                                     split='train')
movies, movies_info = tfds.load('movielens/100k-movies',
                                    with_info=True,
                                    split='train')

In [50]:
# загрузим данные
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

In [51]:
# посмотрим информацию о датасете
movies_info

tfds.core.DatasetInfo(
    name='movielens',
    full_name='movielens/100k-movies/0.1.1',
    description="""
    This dataset contains a set of movie ratings from the MovieLens website, a movie
    recommendation service. This dataset was collected and maintained by [GroupLens]
    (https://grouplens.org/), a research group at the University of Minnesota. There
    are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
    datasets, the movies data and ratings data are joined on "movieId". The 25m
    dataset, latest-small dataset, and 20m dataset contain only movie data and
    rating data. The 1m dataset and 100k dataset contain demographic data in
    addition to movie and rating data.
    
    - "25m": This is the latest stable version of the MovieLens dataset. It is
    recommended for research purposes.
    - "latest-small": This is a small subset of the latest version of the MovieLens
    dataset. It is changed and updated over time by GroupLens.
    - "100

In [52]:
#посмотреть можно так
ds_r = tfds.as_dataframe(ratings.take(5), ratings_info)
ds_r

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7 (Drama),b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4 (doctor/health care),b'doctor',4.0,b'53211'
1,25.0,4 (Comedy)14 (Romance),b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5 (entertainment),b'entertainment',2.0,b'80525'
2,18.0,4 (Comedy),b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17 (student),b'student',4.0,b'55439'
3,50.0,5 (Crime)7 (Drama),b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4 (doctor/health care),b'healthcare',4.0,b'06472'
4,50.0,10 (Horror)16 (Thriller),b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18 (technician/engineer),b'technician',3.0,b'75094'


In [53]:
ds_m = tfds.as_dataframe(movies.take(5), movies_info)
ds_m

,movie_genres,movie_id,movie_title
0,4 (Comedy),b'1681',b'You So Crazy (1994)'
1,4 (Comedy)7 (Drama),b'1457',b'Love Is All There Is (1996)'
2,1 (Adventure)3 (Children),b'500',b'Fly Away Home (1996)'
3,0 (Action),b'838',b'In the Line of Duty 2 (1987)'
4,7 (Drama),b'1648',"b'Niagara, Niagara (1997)'"


In [54]:
# посмотрим набор данных рейтингов
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [55]:
# посмотрим набор данных о фильмах
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


##TensorFlow Recommenders user_id и movie_title. Вариант 1
https://www.tensorflow.org/recommenders/examples/basic_retrieval?hl=ru

In [56]:
# в 1-й моделе сохраним только user_id и movie_title в наборе данных
ratings = ratings.map(lambda x: {'movie_title': x['movie_title'],
                                 'user_id': x['user_id']
})
movies = movies.map(lambda x: x['movie_title'])

In [57]:
# посмотрим
tfds.as_dataframe(ratings.take(5))

,movie_title,user_id
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,b'Strictly Ballroom (1992)',b'92'
2,"b'Very Brady Sequel, A (1996)'",b'301'
3,b'Pulp Fiction (1994)',b'60'
4,b'Scream 2 (1997)',b'197'


In [58]:
# посмотрим
tfds.as_dataframe(movies.take(5))

,
0,b'You So Crazy (1994)'
1,b'Love Is All There Is (1996)'
2,b'Fly Away Home (1996)'
3,b'In the Line of Duty 2 (1987)'
4,"b'Niagara, Niagara (1997)'"


In [59]:
# разобъем данные train и test
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [60]:
# посморим
tfds.as_dataframe(train.take(5)), tfds.as_dataframe(test.take(5))

(                     movie_title user_id
 0         b'Postman, The (1997)'  b'681'
 1             b'Clueless (1995)'  b'442'
 2  b'Maltese Falcon, The (1941)'  b'932'
 3      b'His Girl Friday (1940)'  b'506'
 4            b'Quiz Show (1994)'   b'18',
                        movie_title user_id
 0                b'M*A*S*H (1970)'  b'346'
 1                b'Volcano (1997)'  b'602'
 2  b'2001: A Space Odyssey (1968)'  b'393'
 3     b'Dances with Wolves (1990)'  b'152'
 4                  b'Speed (1994)'  b'738')

In [61]:
# найдем уникальные идентификаторы пользователей и названия фильмов, присутствующие в данных
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# посмотрим
unique_movie_titles[:10], unique_user_ids[:10]

(array([b"'Til There Was You (1997)", b'1-900 (1994)',
        b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
        b'2 Days in the Valley (1996)',
        b'20,000 Leagues Under the Sea (1954)',
        b'2001: A Space Odyssey (1968)',
        b'3 Ninjas: High Noon At Mega Mountain (1998)',
        b'39 Steps, The (1935)'], dtype=object),
 array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
        b'106', b'107'], dtype=object))

Построим модель

In [62]:
# зададим размерность
embedding_dimension = 32

In [63]:
# построим модель пользователей
user_model = tf.keras.Sequential([
             tf.keras.layers.StringLookup(
             vocabulary=unique_user_ids, mask_token=None),
  # добавляем дополнительное встраивание для учета неизвестных токенов
             tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [64]:
# построим модель фильмов
movie_model = tf.keras.Sequential([
              tf.keras.layers.StringLookup(
              vocabulary=unique_movie_titles, mask_token=None),
              tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [65]:
# зададим метрику
metrics = tfrs.metrics.FactorizedTopK(      # обязательно tensorflow==2.15.0
          candidates=movies.batch(128).map(movie_model)
)

In [66]:
# зададим функцию потерь
task = tfrs.tasks.Retrieval(metrics=metrics)

In [67]:
# соберем модель целиком
#сделать это рекомендуется так

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # выбираем пользовательские функции и передаем их в пользовательскую модель
    user_embeddings = self.user_model(features['user_id'])
    # затем выберите элементы фильма и передайте их в модель фильма, возвращая вложения обратно
    positive_movie_embeddings = self.movie_model(features['movie_title'])

    # возвращаем потери и показатели
    return self.task(user_embeddings, positive_movie_embeddings)

In [68]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # установим градиентную ленту для записи градиентов
    with tf.GradientTape() as tape:

      # расчет потерь
      user_embeddings = self.user_model(features['user_id'])
      positive_movie_embeddings = self.movie_model(features['movie_title'])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics['loss'] = loss
    metrics['regularization_loss'] = regularization_loss
    metrics['total_loss'] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # расчет потерь
    user_embeddings = self.user_model(features['user_id'])
    positive_movie_embeddings = self.movie_model(features['movie_title'])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # потерями при регуляризации
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics['loss'] = loss
    metrics['regularization_loss'] = regularization_loss
    metrics['total_loss'] = total_loss

    return metrics

In [93]:
# создадим экземпляр модели
model = MovielensModel(user_model, movie_model)
learning_rate=0.05
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))
type(model)

__main__.MovielensModel

In [94]:
# подготовим данные
cached_train = train.shuffle(100_000).batch(8_192).cache()
cached_test = test.batch(4_096).cache()

In [76]:
# обучим модель
epochs = 16
model.fit(cached_train, epochs=epochs)

Epoch 1/16
10/10 [==============================] - 34s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0050 - factorized_top_k/top_5_categorical_accuracy: 0.0309 - factorized_top_k/top_10_categorical_accuracy: 0.0600 - factorized_top_k/top_50_categorical_accuracy: 0.2228 - factorized_top_k/top_100_categorical_accuracy: 0.3552 - loss: 65254.6676 - regularization_loss: 0.0000e+00 - total_loss: 65254.6676
Epoch 2/16
10/10 [==============================] - 31s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0049 - factorized_top_k/top_5_categorical_accuracy: 0.0319 - factorized_top_k/top_10_categorical_accuracy: 0.0606 - factorized_top_k/top_50_categorical_accuracy: 0.2256 - factorized_top_k/top_100_categorical_accuracy: 0.3589 - loss: 65084.1200 - regularization_loss: 0.0000e+00 - total_loss: 65084.1200
Epoch 3/16
10/10 [==============================] - 32s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0050 - factorized_top_k/top_5_categorical_accuracy: 0.0

In [77]:
# получим предсказания
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 11s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0111 - factorized_top_k/top_50_categorical_accuracy: 0.0896 - factorized_top_k/top_100_categorical_accuracy: 0.1916 - loss: 31565.0306 - regularization_loss: 0.0000e+00 - total_loss: 31565.0306


{'factorized_top_k/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_5_categorical_accuracy': 0.003650000086054206,
 'factorized_top_k/top_10_categorical_accuracy': 0.011099999770522118,
 'factorized_top_k/top_50_categorical_accuracy': 0.089649997651577,
 'factorized_top_k/top_100_categorical_accuracy': 0.19155000150203705,
 'loss': 28655.77734375,
 'regularization_loss': 0,
 'total_loss': 28655.77734375}

In [78]:
# создадим модель
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# рекомендуем фильмы из всего набора
index.index_from_dataset(tf.data.Dataset.zip((movies.batch(100),
                                              movies.batch(100).map(model.movie_model)))
)

# получим рекомендации
_, titles = index(tf.constant(['42']))
print(f'Recommendations for user 42: {titles[0, :5]}')

Recommendations for user 42: [b'Rent-a-Kid (1995)' b'Mirror Has Two Faces, The (1996)' b'Rudy (1993)'
 b'Pinocchio (1940)' b'Michael (1996)']


In [79]:
_

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.1330059, 2.9720678, 2.961512 , 2.8727176, 2.732448 , 2.7109613,
        2.704308 , 2.682961 , 2.6739504, 2.6509192]], dtype=float32)>

##TensorFlow Recommenders . Вариант 2
https://www.tensorflow.org/recommenders/examples/basic_ranking?hl=ru

In [95]:
# загрузим данные
ratings = tfds.load('movielens/100k-ratings', split='train')

ratings = ratings.map(lambda x: {'movie_title': x['movie_title'],
                                 'user_id': x['user_id'],
                                 'user_rating': x['user_rating']
})

In [96]:
# посмотрим
tfds.as_dataframe(ratings.take(5))

,movie_title,user_id,user_rating
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138',4.000000
1,b'Strictly Ballroom (1992)',b'92',2.000000
2,"b'Very Brady Sequel, A (1996)'",b'301',4.000000
3,b'Pulp Fiction (1994)',b'60',4.000000
4,b'Scream 2 (1997)',b'197',3.000000


In [97]:
# разделим на test и train
# случайным образом
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [98]:
# найдем уникальные идентификаторы пользователей и названия фильмов, присутствующие в данных
movie_titles = ratings.batch(1000000).map(lambda x: x['movie_title'])
user_ids = ratings.batch(1000000).map(lambda x: x['user_id'])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [99]:
# посмотрим
unique_movie_titles[:10], unique_user_ids[:10]

(array([b"'Til There Was You (1997)", b'1-900 (1994)',
        b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
        b'2 Days in the Valley (1996)',
        b'20,000 Leagues Under the Sea (1954)',
        b'2001: A Space Odyssey (1968)',
        b'3 Ninjas: High Noon At Mega Mountain (1998)',
        b'39 Steps, The (1935)'], dtype=object),
 array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
        b'106', b'107'], dtype=object))

In [100]:
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers as L

In [101]:
# зададим размерность
embedding_dimension = 32

In [102]:
# построим модель
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    model = tf.keras.Sequential(name='My_model_2')


    # построим эмбэдинг для пользователей
    self.user_embeddings = model.add(L.StringLookup(vocabulary=unique_user_ids,
                                                    mask_token=None),
                                     L.Embedding(len(unique_user_ids) + 1,
                                                    embedding_dimension)
    )

    # построим эмбэдинг для фильмов
    self.movie_embeddings = model.add(L.StringLookup(vocabulary=unique_movie_titles,
                                                     mask_token=None),
                                      L.Embedding(len(unique_movie_titles) + 1,
                                                  embedding_dimension)
    )

    # получим предсказания
    self.ratings = model.add(L.Dense(256, activation='relu'),
                             L.BatchNormalization(),
                             L.Dense(128, activation='relu'),
                             L.BatchNormalization(),
                             L.Dense(64, activation='relu'),
                             L.Dense(1, activation='softmax')
    )

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [103]:
# зададим функцию потерь и метрику
task = tfrs.tasks.Ranking(loss = tf.keras.losses.MeanSquaredError(),
                          metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [104]:
# соберем модель
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features['user_id'], features['movie_title']))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop('user_rating')

    rating_predictions = self(features)

    return self.task(labels=labels, predictions=rating_predictions)

##TensorFlow Recommenders поиск с использованием последовательной модели. Вариант 3
https://tensorflow.google.cn/recommenders/examples/sequential_retrieval?hl=ru

In [5]:
# удалим ранее установленные зависимости
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [6]:
# импортируем библиотеки
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [7]:
# подготовим данные
!wget -nc https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py

--2025-02-05 05:48:06--  https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18041 (18K) [text/plain]
Saving to: ‘example_generation_movielens.py’

example_generation_ 100%[===================>]  17.62K  --.-KB/s    in 0s      

2025-02-05 05:48:06 (85.1 MB/s) - ‘example_generation_movielens.py’ saved [18041/18041]



Данные MovieLens 1M содержит ratings.dat (столбцы: UserID, MovieID, рейтинги, Timestamp) и movies.dat (столбцы: MovieID, Название, Жанры). Пример сценария генерации загружает набор данных размером 1M, принимает оба файла, сохраняет только рейтинги выше 2, формирует временные шкалы взаимодействия с пользователем в фильмах, примеры действий в качестве меток и 10 предыдущих действий пользователя в качестве контекста для прогнозирования

In [8]:
!python -m example_generation_movielens  --data_dir=data/raw  --output_dir=data/examples  --min_timeline_length=3 --max_context_length=10 --max_context_movie_genre_length=10 --min_rating=2 --train_data_fraction=0.9 --build_vocabs=False

2025-02-05 05:48:08.591031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-05 05:48:08.591112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-05 05:48:08.593011: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-05 05:48:10.802048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0205 05:48:13.559783 132989579370496 example_generation_movielens.py:460] Downloading and extracting data.
5917549/5917549 [==============================] - 0s 0us/step
I0205 05:48:14.611369 132989579370496 example_generation_movielens.py:

In [9]:
# задим временные метки фильмов
train_filename = './data/examples/train_movielens_1m.tfrecord'
train = tf.data.TFRecordDataset(train_filename)

test_filename = './data/examples/test_movielens_1m.tfrecord'
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat('Drama', 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    'context_movie_id': tf.strings.as_string(x['context_movie_id']),
    'label_movie_id': tf.strings.as_string(x['label_movie_id'])
})

test_ds = test.map(_parse_function).map(lambda x: {
    'context_movie_id': tf.strings.as_string(x['context_movie_id']),
    'label_movie_id': tf.strings.as_string(x['label_movie_id'])
})

for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_movie_id': array([b'3763', b'3639', b'1101', b'1587', b'1608', b'2628', b'2641',
       b'2006', b'2990', b'2699'], dtype=object),
 'label_movie_id': array([b'1377'], dtype=object)}


In [10]:
train

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [11]:
# составим словарь
movies = tfds.load('movielens/1m-movies', split='train')
movies = movies.map(lambda x: x['movie_id'])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-movies/incomplete.MSXKPB_0.1.1/movielens-train.tfrecord*...: …

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


In [12]:
# зададим размерность
embedding_dimension = 32

# модель запросов
query_model = tf.keras.Sequential([tf.keras.layers.StringLookup(
              vocabulary=unique_movie_ids, mask_token=None),
              tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension),
              tf.keras.layers.GRU(embedding_dimension),
])

# модель кандидатов
candidate_model = tf.keras.Sequential([tf.keras.layers.StringLookup(
              vocabulary=unique_movie_ids, mask_token=None),
              tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
])

In [13]:
# зададим метрики и базовую модель
metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).map(candidate_model))

task = tfrs.tasks.Retrieval(metrics=metrics)

class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features['context_movie_id']
        watch_next_label = features['label_movie_id']

        query_embedding = self._query_model(watch_history)
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [14]:
# определим модель и скомпилируем
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.05))

In [15]:
model

In [16]:
# зададим данные
cached_train = train_ds.shuffle(10_000).batch(12_800).cache()
cached_test = test_ds.batch(2_560).cache()

In [17]:
# обучим
model.fit(cached_train, epochs=16)

Epoch 1/16
67/67 [==============================] - 279s 4s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 111412.1192 - regularization_loss: 0.0000e+00 - total_loss: 111412.1192
Epoch 2/16
67/67 [==============================] - 204s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 104549.6390 - regularization_loss: 0.0000e+00 - total_loss: 104549.6390
Epoch 3/16
67/67 [==============================] - 207s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00

In [18]:
# получим метрики
model.evaluate(cached_test, return_dict=True)

37/37 [==============================] - 59s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0155 - factorized_top_k/top_5_categorical_accuracy: 0.0804 - factorized_top_k/top_10_categorical_accuracy: 0.1379 - factorized_top_k/top_50_categorical_accuracy: 0.3788 - factorized_top_k/top_100_categorical_accuracy: 0.5138 - loss: 15315.2771 - regularization_loss: 0.0000e+00 - total_loss: 15315.2771


{'factorized_top_k/top_1_categorical_accuracy': 0.015533214434981346,
 'factorized_top_k/top_5_categorical_accuracy': 0.08042729645967484,
 'factorized_top_k/top_10_categorical_accuracy': 0.13792258501052856,
 'factorized_top_k/top_50_categorical_accuracy': 0.37876737117767334,
 'factorized_top_k/top_100_categorical_accuracy': 0.5138434171676636,
 'loss': 9323.990234375,
 'regularization_loss': 0,
 'total_loss': 9323.990234375}

In [19]:
index = tfrs.layers.ann.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)
# Получить рекомендации
_, titles = index(tf.constant(['42']))
print(f"Recommendations for user 42: {titles}")


AttributeError: module 'tensorflow_recommenders.layers' has no attribute 'ann'

In [ ]:
_